#Part1 - Environment Setting
This ipynb is written by own our code and the below google drive link can be used to view the whole product, which with many reference codes as well. 

https://drive.google.com/open?id=1IgrvNOUylA7Wt5h5uXTKk8MGPAnnHoLj

The work directory of our own design GAN model code can be found

https://drive.google.com/drive/folders/1uyBz5vGpafZhJqeefS86pgUJnjCtJ3rX?usp=sharing

The final results can be found in the following, included trump to cage train videos in 60s,120s,180s, epochs 21000,30000; trump to cage test videos in in 60s,120s,180s,epochs 21000,30000 etc

https://drive.google.com/drive/folders/1mYbEDYWwjW3ZKnS3IdMrjHCnRZkRb1mZ?usp=sharing


In [1]:
%matplotlib inline
import platform
from tensorflow.python.client import device_lib
import pickle as pkl
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
import random as rand
import glob
import os
import math
import time
from PIL import Image
import cv2
from google.colab import drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [2]:
%cd '/content/gdrive/My Drive/UIUC/2019_spring/ECE549--computer vision/Project/code'
%pwd

/content/gdrive/My Drive/UIUC/2019_spring/ECE549--computer vision/Project/code


'/content/gdrive/My Drive/UIUC/2019_spring/ECE549--computer vision/Project/code'

In [3]:
import keras.backend as K
from keras.models import Model,Sequential,model_from_json
from keras.layers import Input, Dense, Flatten, Reshape, Activation, Dropout
from keras.layers.advanced_activations import LeakyReLU
from keras.layers.convolutional import Conv2D
from keras.optimizers import Adam
import glob
from tqdm import tqdm
!git clone https://gitlab.uip6.com/github/faceswap.git
import faceswap
from faceswap.lib.model.layers import PixelShuffler
!pip install moviepy
!pip install keras_vggface
import imageio
imageio.plugins.ffmpeg.download()
from tensorflow.python.client import device_lib
print(device_lib.list_local_devices())

Using TensorFlow backend.


fatal: destination path 'faceswap' already exists and is not an empty directory.
[name: "/device:CPU:0"
device_type: "CPU"
memory_limit: 268435456
locality {
}
incarnation: 6504864171125407722
, name: "/device:XLA_CPU:0"
device_type: "XLA_CPU"
memory_limit: 17179869184
locality {
}
incarnation: 838318638197015089
physical_device_desc: "device: XLA_CPU device"
, name: "/device:XLA_GPU:0"
device_type: "XLA_GPU"
memory_limit: 17179869184
locality {
}
incarnation: 4568460772123599811
physical_device_desc: "device: XLA_GPU device"
, name: "/device:GPU:0"
device_type: "GPU"
memory_limit: 14800692839
locality {
  bus_id: 1
  links {
  }
}
incarnation: 2637588761371890763
physical_device_desc: "device: 0, name: Tesla T4, pci bus id: 0000:00:04.0, compute capability: 7.5"
]


# Part2 - Face Images Data Processing

##part2-1  Get the face images via upload videos

In [0]:
# from google.colab import files
# # Upload source video
# source_video = files.upload()

# for fn_source_video, _ in source_video.items():
#     print(fn_source_video)
# # Upload target video
# target_video = files.upload()

# for fn_target_video, _ in target_video.items():
#     print(fn_target_video)

In [0]:
# %%capture
# !pip install moviepy
# !pip install keras_vggface
# import imageio
# imageio.plugins.ffmpeg.download()
# fd = MTCNNFaceDetector(sess=K.get_session(), model_path="./mtcnn_weights/")
# !mkdir -p faceA/rgb
# !mkdir -p faceA/binary_mask
# !mkdir -p faceB/rgb
# !mkdir -p faceB/binary_mask

In [0]:
# fn_source_video = 'trump_0.mp4'
# fn_target_video = 'cage_0.mp4' 
# print(str(len(glob.glob("faceA/rgb/*.*"))) + " face(s) extracted from source video: " + fn_source_video + ".")
# print(str(len(glob.glob("faceB/rgb/*.*"))) + " face(s) extracted from target video: " + fn_target_video + ".")
# train_A = glob.glob("faceA/rgb/*.*")
# train_B = glob.glob("faceB/rgb/*.*")

In [7]:
MIN = 0.0
MAX = 255.0
IMAGE_EDGE = 64
IMAGE_DIM = (IMAGE_EDGE, IMAGE_EDGE, 3)

def show_image(image):
  print(np.mean(image))
  print(image)
  imgplot = plt.imshow(image)
  plt.show()

# Normalize image
def preprocess_image(original_image):
  processed_image = original_image - MIN
  processed_image = (processed_image / (MAX - MIN)) * 2.0
  processed_image = processed_image - 1.0
  return processed_image

# Undo normalization
def postprocess_image(processed_image):
  unnormalized = processed_image + 1.0
  unnormalized = (unnormalized / 2.0) * (MAX - MIN)
  unnormalized = unnormalized + MIN
  return unnormalized.astype(int)

# SETUP IMAGES HERE, Images is array [image number, size of image]

fn_source_video = 'trump_0.mp4'
fn_target_video = 'cage_0.mp4' 
print(str(len(glob.glob("faceA/rgb/*.*"))) + " face(s) extracted from source video: " + fn_source_video + ".")
print(str(len(glob.glob("faceB/rgb/*.*"))) + " face(s) extracted from target video: " + fn_target_video + ".")
train_A = glob.glob("faceA/rgb/*.*")
train_B = glob.glob("faceB/rgb/*.*")
images_a = np.array([np.array(Image.open(fname).resize(IMAGE_DIM[0:2])) for fname in train_A])
images_b = np.array([np.array(Image.open(fname).resize(IMAGE_DIM[0:2])) for fname in train_B])
scaled_a = preprocess_image(images_a)
scaled_b = preprocess_image(images_b)

flattendedimages_a = np.array(scaled_a).reshape((scaled_a.shape[0], -1))
flattendedimages_b = np.array(scaled_b).reshape((scaled_b.shape[0], -1))
# Size of input images to discriminators (SIZE OF IMAGE)
assert(flattendedimages_a.shape[1] == flattendedimages_b.shape[1])
input_size = flattendedimages_a.shape[1]

# Scale images to fit functions
scaledimages_a = flattendedimages_a.reshape((-1,)+IMAGE_DIM)
scaledimages_b = flattendedimages_b.reshape((-1,)+IMAGE_DIM)

708 face(s) extracted from source video: trump_0.mp4.
647 face(s) extracted from target video: cage_0.mp4.


##part2-2  Plot and save fake face images

In [0]:
def plot_generated_images(epoch, generator, input_images, dim=(10,10), figsize=(10,10), label='a'):
    generated_images = generator.predict(input_images)
    generated_images = postprocess_image(generated_images.reshape(input_images.shape))
    plt.figure(figsize=figsize)
    for i in range(input_images.shape[0]):
        plt.subplot(dim[0], dim[1], i+1)
        plt.imshow(postprocess_image(input_images[i]), interpolation='nearest')
        plt.axis('off')
    plt.tight_layout()
    plt.savefig('input_' + label + '_generated_image_%d.png' %epoch)
    for i in range(generated_images.shape[0]):
        plt.subplot(dim[0], dim[1], i+1)
        plt.imshow(generated_images[i], interpolation='nearest')
        plt.axis('off')
    plt.tight_layout()
    plt.savefig('gan_' + label + '_generated_image_%d.png' %epoch)

# Part3 - Build the GAN model 

In [0]:
def create_encoder(latent_size, alpha=0.1):
  
  inputs = Input(shape=IMAGE_DIM)
  
  encoder = Reshape(IMAGE_DIM)(inputs)
  
  encoder = Conv2D(IMAGE_EDGE * 2, kernel_size=5, strides=2, padding='same')(encoder)
  encoder = LeakyReLU(alpha)(encoder)
  encoder = Conv2D(IMAGE_EDGE * 4, kernel_size=5, strides=2, padding='same')(encoder)
  encoder = LeakyReLU(alpha)(encoder)
  encoder = Conv2D(IMAGE_EDGE * 8, kernel_size=5, strides=2, padding='same')(encoder)
  encoder = LeakyReLU(alpha)(encoder)
  encoder = Conv2D(IMAGE_EDGE * 16, kernel_size=5, strides=2, padding='same')(encoder)
  encoder = LeakyReLU(alpha)(encoder)

  encoder = Flatten()(encoder)
  encoder = Dense(latent_size)(encoder)
  encoder = Dense(4 * 4 * 1024)(encoder)
  encoder = Reshape((4, 4, 1024))(encoder)
  
  encoder = Conv2D(512 * 4, kernel_size=3, padding='same')(encoder)
  encoder = LeakyReLU(alpha)(encoder)
  encoder = PixelShuffler()(encoder)
  
  model = Model(inputs=[inputs],outputs=[encoder])
  
  return model

def create_decoder(out_dim, alpha=0.1):
  
  inputs = Input(shape=(4, 4, 512 * 4))
  
  decoder = Reshape((8, 8, 512))(inputs)

  decoder = Conv2D(256 * 4, kernel_size=3, padding='same')(decoder)
  decoder = LeakyReLU(alpha)(decoder)
  decoder = PixelShuffler()(decoder)
  
  decoder = Conv2D(128 * 4, kernel_size=3, padding='same')(decoder)
  decoder = LeakyReLU(alpha)(decoder)
  decoder = PixelShuffler()(decoder)
  
  decoder = Conv2D(64 * 4, kernel_size=3, padding='same')(decoder)
  decoder = LeakyReLU(alpha)(decoder)
  decoder = PixelShuffler()(decoder)
  
  decoder = Conv2D(3, kernel_size=5, padding='same', activation='sigmoid')(decoder)
  
  decoder = Flatten()(decoder)
  decoder = Dense(out_dim)(decoder)
  decoder = Dense(64 * 64 * 3)(decoder)
  decoder = Activation('tanh')(decoder)
  decoder = Reshape(IMAGE_DIM)(decoder)
  
  model = Model(inputs=[inputs],outputs=[decoder])
  
  return model

def create_discriminator(alpha=0.1, dropout=0.6):
  
  inputs = Input(shape=IMAGE_DIM)
  
  discriminator = Reshape(IMAGE_DIM)(inputs)

  discriminator = Conv2D(IMAGE_EDGE, kernel_size=5, strides=2, padding='same')(discriminator)
  discriminator = LeakyReLU(alpha)(discriminator)
  discriminator = Dropout(dropout)(discriminator)
  
  discriminator = Conv2D(IMAGE_EDGE, kernel_size=5, strides=1, padding='same')(discriminator)
  discriminator = LeakyReLU(alpha)(discriminator)
  discriminator = Dropout(dropout)(discriminator)
  
  discriminator = Conv2D(IMAGE_EDGE, kernel_size=5, strides=1, padding='same')(discriminator)
  discriminator = LeakyReLU(alpha)(discriminator)
  discriminator = Dropout(dropout)(discriminator)
  
  discriminator = Flatten()(discriminator)
  discriminator = Dense(IMAGE_EDGE * 2)(discriminator)
  discriminator = LeakyReLU(alpha)(discriminator)
  discriminator = Dense(1)(discriminator)
  discriminator = Activation('sigmoid')(discriminator)
  
  model = Model(inputs=[inputs],outputs=[discriminator])
  
  return model


 # Part4 - Train function,loss and optimizers

In [0]:
# Loss functions https://github.com/shaoanlu/faceswap-GAN/blob/87146b5995a41363f1d93e32c7c8338b78f80b11/networks/losses.py
def first_order(x, axis=1):
  img_nrows = x.shape[1]
  img_ncols = x.shape[2]
  if axis == 1:
    return K.abs(x[:, 0:img_nrows - 1, 0:img_ncols - 1, :] - x[:, 1:img_nrows, 0:img_ncols - 1, :])
  elif axis == 2:
    return K.abs(x[:, 0:img_nrows - 1, 0:img_ncols - 1, :] - x[:, 0:img_nrows - 1, 1:img_ncols, :])
  else:
    return None  

def calc_loss(pred, target, loss='l2'):
  if loss.lower() == "l2":
    return K.mean(K.square(pred - target))
  elif loss.lower() == "l1":
    return K.mean(K.abs(pred - target))
  elif loss.lower() == "cross_entropy":
    return -K.mean(K.log(pred + K.epsilon())*target + K.log(1 - pred + K.epsilon())*(1 - target))
  else:
    raise ValueError(f'Recieve an unknown loss type: {loss}.')
    
def encoding_loss(y_true,y_pred):
  reconstruction_loss = calc_loss(y_pred, y_true, "l1")
  row_pred_consist = K.abs(y_pred[:, 0:64 - 1, 0:64 - 1, :] - y_pred[:, 1:64, 0:64 - 1, :])
  row_true_consist = K.abs(y_true[:, 0:64 - 1, 0:64 - 1, :] - y_true[:, 1:64, 0:64 - 1, :])
  col_pred_consist = K.abs(y_pred[:, 0:64 - 1, 0:64 - 1, :] - y_pred[:, 0:64 - 1, 1:64, :])
  col_true_consist = K.abs(y_true[:, 0:64 - 1, 0:64 - 1, :] - y_true[:, 0:64 - 1, 1:64, :])
  edge_loss_y = calc_loss(row_pred_consist, row_true_consist, "l1")
  edge_loss_x = calc_loss(col_pred_consist, col_true_consist, "l1")
  generator_loss = reconstruction_loss + edge_loss_y + edge_loss_x
  return generator_loss

In [0]:
def create_models():
  K.clear_session()
  
  # Creating GAN models
  inputs = Input(shape=IMAGE_DIM)
  
  gen_opt_a = Adam(lr=10e-3)
  gen_opt_b = Adam(lr=10e-3)
  encoder = create_encoder(1024)
  decoder_a = create_decoder(1024)
  decoder_b = create_decoder(1024)
  generator_a = Model(inputs, decoder_a(encoder(inputs)))
  generator_b = Model(inputs, decoder_b(encoder(inputs)))
  generator_a.compile(loss='binary_crossentropy', optimizer=gen_opt_a)
  generator_b.compile(loss='binary_crossentropy', optimizer=gen_opt_b)
  
  disc_opt_a = Adam(lr=10e-4)
  disc_opt_b = Adam(lr=10e-4)
  discriminator_a = create_discriminator()
  discriminator_b = create_discriminator()
  discriminator_a.compile(loss='binary_crossentropy', optimizer=disc_opt_a)
  discriminator_b.compile(loss='binary_crossentropy', optimizer=disc_opt_b)
  
  def make_trainable(net, val):
    net.trainable = val
    for l in net.layers:
        l.trainable = val
        
  make_trainable(discriminator_a, False)
  make_trainable(discriminator_b, False)
  fake_a = generator_a(inputs)
  fake_b = generator_b(inputs)
  validity_a = discriminator_a(fake_a)
  validity_b = discriminator_b(fake_b)
  GAN_a = Model(inputs, [fake_a, validity_a])
  GAN_b = Model(inputs, [fake_b, validity_b])
  GAN_a.compile(loss=[encoding_loss, 'binary_crossentropy'], optimizer=gen_opt_a, loss_weights=[1,0.1])
  GAN_b.compile(loss=[encoding_loss, 'binary_crossentropy'], optimizer=gen_opt_b, loss_weights=[1,0.1])
  return generator_a, generator_b, discriminator_a, discriminator_b, GAN_a, GAN_b

In [0]:
models_dir = "./models" # Path to saved model weights

In [0]:
def training(generator_a, generator_b, discriminator_a, discriminator_b, GAN_a, GAN_b, imageset_a, imageset_b, past_epochs, epochs=100, batch_size=128):
  for e in range(1,epochs+1 ):
    np.random.shuffle(imageset_a)
    np.random.shuffle(imageset_b)
    for ii in range(min(imageset_a.shape[0]//batch_size, imageset_b.shape[0]//batch_size)):
      # image inputs
      batch_a = np.array(imageset_a[ii*batch_size:(ii+1)*batch_size])
      batch_b = np.array(imageset_b[ii*batch_size:(ii+1)*batch_size])

      # Generate fake images from other image inputs
      generated_images_a = generator_a.predict(batch_b)
      generated_images_b = generator_b.predict(batch_a)

      #Construct different batches of  real and fake data 
      X_a = np.concatenate([batch_a, generated_images_a])
      X_b = np.concatenate([batch_b, generated_images_b])
      # Labels for generated and real data
      y_dis=np.zeros(2*batch_size)
      y_dis[:batch_size]=0.9

      #Pre train discriminator on  fake and real data  before starting the gan. 
      discriminator_a.trainable=True
      discriminator_a.train_on_batch(X_a, y_dis)
      discriminator_b.trainable=True
      discriminator_b.train_on_batch(X_b, y_dis)

      y_gen = np.ones(batch_size)

      # During the training of gan, 
      # the weights of discriminator should be fixed. 
      #We can enforce that by setting the trainable flag
      discriminator_a.trainable=False
      discriminator_b.trainable=False

      #training  the GAN by alternating the training of the Discriminator 
      #and training the chained GAN model with Discriminator’s weights freezed.
      GAN_a.train_on_batch(batch_b, [batch_a, y_gen])
      GAN_b.train_on_batch(batch_a, [batch_b, y_gen])

    if e % 10 == 0:
      generator_a_json = generator_a.to_json()
      with open(models_dir + "/generator_a.json", "w") as json_file:
          json_file.write(generator_a_json)
      generator_a.save_weights(models_dir + "/generator_a.h5")
      generator_b_json = generator_b.to_json()
      with open(models_dir + "/generator_b.json", "w") as json_file:
          json_file.write(generator_b_json)
      generator_b.save_weights(models_dir + "/generator_b.h5")
      discriminator_a_json = discriminator_a.to_json()
      with open(models_dir + "/discriminator_a.json", "w") as json_file:
          json_file.write(discriminator_a_json)
      discriminator_a.save_weights(models_dir + "/discriminator_a.h5")
      discriminator_b_json = discriminator_b.to_json()
      with open(models_dir + "/discriminator_b.json", "w") as json_file:
          json_file.write(discriminator_b_json)
      discriminator_b.save_weights(models_dir + "/discriminator_b.h5")
      GAN_a_json = GAN_a.to_json()
      with open(models_dir + "/GAN_a.json", "w") as json_file:
          json_file.write(GAN_a_json)
      GAN_a.save_weights(models_dir + "/GAN_a.h5")
      GAN_b_json = GAN_b.to_json()
      with open(models_dir + "/GAN_b.json", "w") as json_file:
          json_file.write(GAN_b_json)
      GAN_b.save_weights(models_dir + "/GAN_b.h5")
      if e % 1000 == 0:
      # image inputs
       batch_a = np.array(imageset_a[0:batch_size])
       batch_b = np.array(imageset_b[0:batch_size])
       plot_generated_images(e + past_epochs, generator_a, batch_b, label='a')
       plot_generated_images(e + past_epochs, generator_b, batch_a, label='b')
       plot_generated_images(e + past_epochs, generator_a, batch_a, label='a_true')
       plot_generated_images(e + past_epochs, generator_b, batch_b, label='b_true')
  return generator_a, generator_b, discriminator_a, discriminator_b, GAN_a, GAN_b 

In [0]:
reuse = False
if not reuse:
  generator_a, generator_b, discriminator_a, discriminator_b, GAN_a, GAN_b = create_models()
# else:
#   json_file = open(models_dir + '/generator_a.json', 'r')
#   loaded_model_json = json_file.read()
#   json_file.close()
#   generator_a = model_from_json(loaded_model_json)
#   generator_a.load_weights(models_dir + "/generator_a.h5")
  
#   json_file = open( models_dir + '/generator_b.json', 'r')
#   loaded_model_json = json_file.read()
#   json_file.close()
#   generator_b = model_from_json(loaded_model_json)
#   generator_b.load_weights(models_dir + "/generator_b.h5")
  
#   json_file = open(models_dir + '/discriminator_a.json', 'r')
#   loaded_model_json = json_file.read()
#   json_file.close()
#   discriminator_a = model_from_json(loaded_model_json)
#   discriminator_a.load_weights(models_dir + "/discriminator_a.h5")
  
#   json_file = open(models_dir + '/discriminator_b.json', 'r')
#   loaded_model_json = json_file.read()
#   json_file.close()
#   discriminator_b = model_from_json(loaded_model_json)
#   discriminator_b.load_weights(models_dir + "/discriminator_b.h5")
  
#   json_file = open(models_dir + '/GAN_a.json', 'r')
#   loaded_model_json = json_file.read()
#   json_file.close()
#   GAN_a = model_from_json(loaded_model_json)
#   GAN_a.load_weights(models_dir + "/GAN_a.h5")
  
#   json_file = open(models_dir + '/GAN_b.json', 'r')
#   loaded_model_json = json_file.read()
#   json_file.close()
#   GAN_b = model_from_json(loaded_model_json)
#   GAN_b.load_weights(models_dir + "/GAN_b.h5")

#Part5 - Face Image Training and Testing 

In [0]:
epochs = 5000
batch_size = 100
past_epochs = 4000
generator_a, generator_b, discriminator_a, discriminator_b, GAN_a, GAN_b = training(generator_a, generator_b, discriminator_a, discriminator_b, GAN_a, GAN_b, scaledimages_a, scaledimages_b, past_epochs, epochs, batch_size)

/usr/local/lib/python3.6/dist-packages/keras/engine/training.py:490: UserWarning: Discrepancy between trainable weights and collected trainable weights, did you set `model.trainable` without calling `model.compile` after ?
  'Discrepancy between trainable weights and collected trainable'
/usr/local/lib/python3.6/dist-packages/matplotlib/figure.py:98: MatplotlibDeprecationWarning: 
Adding an axes using the same arguments as a previous axes currently reuses the earlier instance.  In a future version, a new instance will always be created and returned.  Meanwhile, this warning can be suppressed, and the future behavior ensured, by passing a unique label to each axes instance.
  "Adding an axes using the same arguments as a previous axes "


In [0]:
global graph
graph = tf.get_default_graph() 
with graph.as_default():
  random_input = np.random.rand(1,64,64,3)*2 - 1
  show_image(postprocess_image(generator_a.predict(random_input).reshape(64,64,3)))

# Part6 - Images convert back into Video

Note: after train, this part can be used as either train videos or test videos

This part can be done using reference code  as well, which will be more suitable code to convert images into videos

All we need to do is give the h5 format weights to the gan model

## Part6-1 trained weights to face detector

In [0]:
from converter.video_converter import VideoConverter
from networks.faceswap_gan_model import FaceswapGANModel
global model, vggface
global train_batchA, train_batchB
# del model
# del vggface
# del train_batchA
# del train_batchB
tf.reset_default_graph()
K.clear_session()
# models_dir = "./models" # Path to saved model weights
# model = FaceswapGANModel(**arch_config)
# model.load_weights(path=models_dir)

In [0]:
# fd = MTCNNFaceDetector(sess=K.get_session(), model_path="./mtcnn_weights/")
# vc = VideoConverter()
# vc.set_face_detector(fd)
# vc.set_gan_model(model)

In [0]:
options = {
    # ===== Fixed =====
    "use_smoothed_bbox": True,
    "use_kalman_filter": True,
    "use_auto_downscaling": False,
    "bbox_moving_avg_coef": 0.65,
    "min_face_area": 35 * 35,
    "IMAGE_SHAPE": (64,64,3), #model.IMAGE_SHAPE
    # ===== Tunable =====
    "kf_noise_coef": 1e-3,
    "use_color_correction": "hist_match",
    "detec_threshold": 0.8,
    "roi_coverage": 0.9,
    "enhance": 0.,
    "output_type": 3,
    "direction": "AtoB", # ==================== This line determines the transform direction ====================
}

## Part6-2 input test video cases

In [0]:
fn_source_video = 'trump_0.mp4'
fn_target_video = 'cage_0.mp4'

# fn_source_video = 'cage_0.mp4'
# fn_target_video = 'trump_0.mp4'

# fn_source_video = 'trump_1.mp4'
# fn_target_video = 'cage_1.mp4'

# fn_source_video = 'trump_2.mp4'
# fn_target_video = 'cage_0.mp4'

# fn_source_video = 'trump3_0.mp4'
# fn_target_video = 'cage_0.mp4'

# fn_source_video = 'trump3_1.mp4'
# fn_target_video = 'cage_0.mp4'

# fn_source_video = 'trump3_2.mp4'
# fn_target_video = 'cage_0.mp4'

if options["direction"] == "AtoB":
    input_fn = fn_source_video
    output_fn = "OUTPUT_VIDEO_AtoB.mp4"
elif options["direction"] == "BtoA":
    input_fn = fn_target_video
    output_fn = "OUTPUT_VIDEO_BtoA.mp4"

duration = None # None or a non-negative float tuple: (start_sec, end_sec). Duration of input video to be converted

In [0]:
# vc.convert(input_fn=input_fn, output_fn=output_fn, options=options, duration=duration)

## Part6-3 download output videos

In [0]:
# from google.colab import files
# if options["direction"] == "AtoB":
#     files.download("OUTPUT_VIDEO_AtoB.mp4")
# elif options["direction"] == "BtoA":
#     files.download("OUTPUT_VIDEO_BtoA.mp4")